# metadata

> Functions that facilitate retrieval of metadata from data sources available from this package

In [ ]:
#| default_exp metadata

In [ ]:
#| export
from __future__ import annotations
from typing import Literal
from importlib import import_module
from inspect import signature

import pandas as pd
from  thefuzz import process, fuzz

from fastcore.script import call_parse

from finsets import RESOURCES

In [ ]:
#| exports 
METADATA_ALL = RESOURCES/'metadata_all.pkl'
METADATA_RAW = RESOURCES/'metadata_raw.pkl'
METADATA_FEATURES = RESOURCES/'metadata_features.pkl'

In [ ]:
#| export
def list_datasets(submodules: list=['wrds', 'papers'] # list of submodules to collect metadata from
                      ) -> pd.DataFrame:
    "Go through `submodules` of `finsets` and print their names out"
    
    for name in submodules:
        module = import_module(f'finsets.{name}')
        for sub in dir(module):
            if sub.startswith('_'): continue
            print(f'{name}.{sub}')

In [ ]:
list_datasets()

wrds.compa
wrds.compq
wrds.crspd
wrds.crspm
wrds.ibes_ltg
wrds.linking
wrds.ratios
wrds.wrds_api
papers.hassan_etal_2019


In [ ]:
#| export
def features_metadata(submodules: list=['wrds', 'papers'] # list of submodules to collect metadata from
                      ) -> pd.DataFrame:
    "Go through `submodules` of `finsets` and collect metadata from all functions that have `return_metadata` parameter"
    
    df = pd.DataFrame(columns=['name','label','output_of','inputs','inputs_generated_by'])
    for name in submodules:
        module = import_module(f'finsets.{name}')
        for sub in dir(module):
            if sub.startswith('_'): continue
            submodule = import_module(f'finsets.{name}.{sub}')
            for func_name in submodule.__all__:
                func = getattr(submodule, func_name)
                if callable(func):
                    try: 
                        params = signature(func).parameters
                    except:
                        continue
                    if 'return_metadata' in params: 
                        meta = func(return_metadata=True)
                        for var_name in meta['outputs']:
                            for input_name in meta['inputs']:    
                                new_meta = pd.DataFrame({'name':var_name, 
                                                'label':meta['labels'][var_name], 
                                                'output_of':f'{name}.{sub}.{func_name}', 
                                                'inputs':','.join(meta['inputs'][input_name]),
                                                'inputs_generated_by':input_name}, index=[0])
                                df = pd.concat([df,new_meta],ignore_index=True)
    return df

In [ ]:
features_metadata()

,name,label,output_of,inputs,inputs_generated_by
0,bookeq,Book equity,wrds.compa.book_equity,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb",wrds.compa.clean
1,shreq,Shareholder equity,wrds.compa.book_equity,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb",wrds.compa.clean
2,pref_stock,Preferred stock,wrds.compa.book_equity,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb",wrds.compa.clean
3,tobinq,Tobin Q,wrds.compa.tobin_q,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb,pr...",wrds.compa.clean
4,equityiss_tot,Equity issuance,wrds.compa.issuance_vars,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb,ss...",wrds.compa.clean
5,equityiss_cfs,Equity issuance,wrds.compa.issuance_vars,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb,ss...",wrds.compa.clean
6,debtiss_tot,Debt issuance,wrds.compa.issuance_vars,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb,ss...",wrds.compa.clean
7,debtiss_cfs,Debt issuance,wrds.compa.issuance_vars,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb,ss...",wrds.compa.clean
8,debtiss_bs,Debt issuance,wrds.compa.issuance_vars,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb,ss...",wrds.compa.clean
9,ppentpch,Pct change in net PPE,wrds.compa.investment_vars,"ppent,capx,at",wrds.compa.clean


In [ ]:
#| export
def raw_metadata(submodules=['wrds', 'papers'] # list of submodules to collect metadata from
                ) -> pd.DataFrame:
    "Go through `submodules` of `finsets` and collect metadata from `raw_metadata` functions (if present)"

    df = pd.DataFrame(columns=['name','label','output_of','type'])
    for name in submodules:
        module = import_module(f'finsets.{name}')
        for sub in dir(module):
            if sub.startswith('_'): continue
            submodule = import_module(f'finsets.{name}.{sub}')
            if 'raw_metadata' in submodule.__all__:
                df = pd.concat([df,submodule.raw_metadata()],ignore_index=True)
    return df

In [ ]:
raw_metadata()

Loading library list...
Done
Approximately 881760 rows in comp.funda.
Loading library list...
Done
Approximately 4922867 rows in crsp.msf.
Approximately 111623 rows in crsp.msenames.
Loading library list...
Done
Approximately 31823580 rows in ibes.detu_epsus.
Loading library list...
Done
Approximately 2750800 rows in wrdsapps_finratio_ibes.firm_ratio_ibes.


,name,label,output_of,type,nr_rows,wrds_library,wrds_table,group
0,gvkey,Global Company Key,wrds.compa.download,VARCHAR(6),881760,comp,funda,<NA>
1,datadate,<NA>,wrds.compa.download,DATE,881760,comp,funda,<NA>
2,fyear,Data Year - Fiscal,wrds.compa.download,DOUBLE_PRECISION,881760,comp,funda,<NA>
3,indfmt,<NA>,wrds.compa.download,VARCHAR(12),881760,comp,funda,<NA>
4,consol,<NA>,wrds.compa.download,VARCHAR(2),881760,comp,funda,<NA>
...,...,...,...,...,...,...,...,...
1106,ffi48,<NA>,wrds.ratios.download,DOUBLE_PRECISION,2750800,wrdsapps_finratio_ibes,firm_ratio_ibes,<NA>
1107,ffi49_desc,<NA>,wrds.ratios.download,VARCHAR(5),2750800,wrdsapps_finratio_ibes,firm_ratio_ibes,<NA>
1108,ffi49,<NA>,wrds.ratios.download,DOUBLE_PRECISION,2750800,wrdsapps_finratio_ibes,firm_ratio_ibes,<NA>
1109,ticker,EXCHANGE TICKER SYMBOL - HISTORICAL,wrds.ratios.download,VARCHAR(8),2750800,wrdsapps_finratio_ibes,firm_ratio_ibes,ID


In [ ]:
#| export
def all_metadata(submodules=['wrds', 'papers'] # list of submodules to collect metadata from
                ) -> pd.DataFrame:
    "Collects `raw_metadata` and `features_metadata` from `submodules` of `finsets`"

    df =  pd.concat([features_metadata(submodules), raw_metadata(submodules)], ignore_index=True)
    df.to_pickle(METADATA_ALL)
    return df 

In [ ]:
meta = all_metadata()

Loading library list...
Done
Approximately 881760 rows in comp.funda.
Loading library list...
Done
Approximately 4922867 rows in crsp.msf.
Approximately 111623 rows in crsp.msenames.
Loading library list...
Done
Approximately 31823580 rows in ibes.detu_epsus.
Loading library list...
Done
Approximately 2750800 rows in wrdsapps_finratio_ibes.firm_ratio_ibes.


In [ ]:
#| export
@call_parse
def search(query: str,              # What to search for 
           meta: str='all',   #"all", "features", or "raw"; specifies the function that fetches the metadata you want to search through
           field: str='label', # Which column in the metadata table you want to search through. Use "name" to search variable names.  
           limit: int=10,           # How many results to display                      
           ) -> pd.DataFrame:
    "Search for `query` in metadata returned by f`{meta_func}_metadata`; return `limit` number of results"

    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 100)

    try:   
        #import finsets
        metadata = pd.read_pickle(METADATA_ALL)  #getattr(finsets, f'{meta}_metadata')()
        results = process.extractBests(query, metadata[field].dropna().astype('string'), 
                                        scorer = fuzz.token_sort_ratio,
                                        limit=limit)
        rows = [x[2] for x in results]
        scores = [x[1] for x in results]
        df = metadata.iloc[rows]
        df.index = scores
        df.index.name="SCORE"
        df.columns = df.columns.str.upper()
    finally:
        pd.reset_option('display.max_columns')
        pd.reset_option('display.width')        
    return df
       

In [ ]:
meta.label.astype('string')

0                                Book equity
1                         Shareholder equity
2                            Preferred stock
3                                    Tobin Q
4                            Equity issuance
                        ...                 
1125                                    <NA>
1126                                    <NA>
1127                                    <NA>
1128    EXCHANGE TICKER SYMBOL - HISTORICAL 
1129          CUSIP IDENTIFIER - HISTORICAL 
Name: label, Length: 1130, dtype: string

In [ ]:
search('tobins q')

,NAME,LABEL,OUTPUT_OF,INPUTS,INPUTS_GENERATED_BY,TYPE,NR_ROWS,WRDS_LIBRARY,WRDS_TABLE,GROUP
SCORE,,,,,,,,,,
93,tobinq,Tobin Q,wrds.compa.tobin_q,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb,pr...",wrds.compa.clean,<NA>,<NA>,<NA>,<NA>,<NA>
50,aqc,Acquisitions,wrds.compa.download,NaN,NaN,DOUBLE_PRECISION,881760,comp,funda,<NA>
45,acqlntal,Acquired Loans,wrds.compa.download,NaN,NaN,DOUBLE_PRECISION,881760,comp,funda,<NA>
43,optgr,Options - Granted,wrds.compa.download,NaN,NaN,DOUBLE_PRECISION,881760,comp,funda,<NA>
43,pvon,Provisions - Other (Net),wrds.compa.download,NaN,NaN,DOUBLE_PRECISION,881760,comp,funda,<NA>
42,bid,Closing Bid,wrds.crspm.download,NaN,NaN,DOUBLE_PRECISION,4922867,crsp,msf,<NA>
42,ask,Closing Ask,wrds.crspm.download,NaN,NaN,DOUBLE_PRECISION,4922867,crsp,msf,<NA>
40,optca,Options - Cancelled (-),wrds.compa.download,NaN,NaN,DOUBLE_PRECISION,881760,comp,funda,<NA>
40,optexd,Options - Exercised (-),wrds.compa.download,NaN,NaN,DOUBLE_PRECISION,881760,comp,funda,<NA>


In [ ]:
search('total assets', meta = 'raw', limit=5)

,NAME,LABEL,OUTPUT_OF,INPUTS,INPUTS_GENERATED_BY,TYPE,NR_ROWS,WRDS_LIBRARY,WRDS_TABLE,GROUP
SCORE,,,,,,,,,,
100,at,Assets - Total,wrds.compa.download,NaN,NaN,DOUBLE_PRECISION,881760,comp,funda,<NA>
75,act,Current Assets - Total,wrds.compa.download,NaN,NaN,DOUBLE_PRECISION,881760,comp,funda,<NA>
75,ao,Assets - Other,wrds.compa.download,NaN,NaN,DOUBLE_PRECISION,881760,comp,funda,<NA>
71,batr,Benefits Assumed - Total,wrds.compa.download,NaN,NaN,DOUBLE_PRECISION,881760,comp,funda,<NA>
69,dptb,Deposits - Total - Banks,wrds.compa.download,NaN,NaN,DOUBLE_PRECISION,881760,comp,funda,<NA>


In [ ]:
search('cash flows', meta = 'features')

,NAME,LABEL,OUTPUT_OF,INPUTS,INPUTS_GENERATED_BY,TYPE,NR_ROWS,WRDS_LIBRARY,WRDS_TABLE,GROUP
SCORE,,,,,,,,,,
72,pcf,Price/Cash flow,wrds.ratios.download,NaN,NaN,DOUBLE_PRECISION,2750800,wrdsapps_finratio_ibes,firm_ratio_ibes,Valuation
69,scf,Cash Flow Format,wrds.compa.download,NaN,NaN,DOUBLE_PRECISION,881760,comp,funda,<NA>
69,cfm,Cash Flow Margin,wrds.ratios.download,NaN,NaN,DOUBLE_PRECISION,2750800,wrdsapps_finratio_ibes,firm_ratio_ibes,Financial_Soundness
60,cash_debt,Cash Flow/Total Debt,wrds.ratios.download,NaN,NaN,DOUBLE_PRECISION,2750800,wrdsapps_finratio_ibes,firm_ratio_ibes,Financial_Soundness
60,cash_ratio,Cash Ratio,wrds.ratios.download,NaN,NaN,DOUBLE_PRECISION,2750800,wrdsapps_finratio_ibes,firm_ratio_ibes,Liquidity
59,txdc,Deferred Taxes (Cash Flow),wrds.compa.download,NaN,NaN,DOUBLE_PRECISION,881760,comp,funda,<NA>
57,ch,Cash,wrds.compa.download,NaN,NaN,DOUBLE_PRECISION,881760,comp,funda,<NA>
57,ask,Closing Ask,wrds.crspm.download,NaN,NaN,DOUBLE_PRECISION,4922867,crsp,msf,<NA>
54,cflow2la_is,Cash flows to lagged assets,wrds.compa.cashflow_vars,"dtdate,oancf,ib,dp,at",wrds.compa.clean,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
search('txditc', field='name', limit=5)

,NAME,LABEL,OUTPUT_OF,INPUTS,INPUTS_GENERATED_BY,TYPE,NR_ROWS,WRDS_LIBRARY,WRDS_TABLE,GROUP
SCORE,,,,,,,,,,
100,txditc,Deferred Taxes and Investment Tax Credit,wrds.compa.download,NaN,NaN,DOUBLE_PRECISION,881760,comp,funda,<NA>
80,txdc,Deferred Taxes (Cash Flow),wrds.compa.download,NaN,NaN,DOUBLE_PRECISION,881760,comp,funda,<NA>
80,txdi,Income Taxes - Deferred,wrds.compa.download,NaN,NaN,DOUBLE_PRECISION,881760,comp,funda,<NA>
67,tic,Ticker Symbol,wrds.compa.download,NaN,NaN,VARCHAR(8),881760,comp,funda,<NA>
67,tdc,Deferred Income Taxes - Net (Cash Flow),wrds.compa.download,NaN,NaN,DOUBLE_PRECISION,881760,comp,funda,<NA>


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()